In [ ]:
import cv2
import numpy as np

# Codigo de exemplo para extrair uma pessoa do fundo usando contornos e morfologia

# Open webcam
vidBuffer = cv2.VideoCapture(0)
framerate = 60
delayFrame = int(1.0 / framerate * 1000)

while True:
    flag, img = vidBuffer.read()
    if not flag:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Use Otsu threshold for automatic separation
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations to clean noise & fill holes
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Build mask
    mask = np.zeros(img.shape[:2], np.uint8)
    for cnt in contours:
        if cv2.contourArea(cnt) > 5000:  # ignore tiny blobs
            cv2.drawContours(mask, [cnt], -1, 255, thickness=-1)

    mask = cv2.bitwise_not(mask)  # Invert mask


    # Invert mask
    mask_inv = cv2.bitwise_not(mask)

    # Extract person
    fg = cv2.bitwise_and(img, img, mask=mask)
    
    cv2.imshow("Person", fg)

    key = cv2.waitKey(delayFrame)
    if key == 27:  # ESC to exit
        break

vidBuffer.release()
cv2.destroyAllWindows()


In [ ]:
def AugmentedReality(video0, ini, fin): 
    # Abre o vídeo original
    original = cv2.VideoCapture(video0)
    
    # Prepara o vídeo cortado
    cut_name = "cut_video.avi"
    height = int(original.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(original.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    # Define o codec e cria o VideoWriter para o vídeo cortado
    videoWriter = cv2.VideoWriter(cut_name, -1, original.get(cv2.CAP_PROP_FPS), (width, height), 0)
    
    # Cortar o video entre os frames ini e fin
    for i in range(original.get(cv2.CAP_PROP_FRAME_COUNT)):
        ret, frame = original.read()
        if not ret:
            break
        if i >= ini and i <= fin:
            videoWriter.write(frame)
    """
    Common Issues:
    Codec not supported: If the specified codec is not supported by your system, you may see an error message. 
    In this case, you can try using a different codec or adjusting the fourcc parameter.
    Insufficient memory: If the video is too large, you may run out of memory. To mitigate this issue, you can try 
    reducing the frame rate or using a more efficient codec.
    Incorrect frame rate: If the frame rate is incorrect, you may see a jerky or choppy video. To fix this issue, 
    you can adjust the fps parameter or use a different codec.
    """
    
    # Para a deteção e classificação de partes do corpo temos 3 opcoes:
    # 1 - Usar HOG para detetar a pessoa e extrair o braco (https://thedatafrog.com/en/articles/human-detection-video/)
    # 2 - Usar o haarcascade para detetar a pessoa (https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html)
    # 3 - usar outra biblioteca (mediapipe, openpose)
    
    # Iniciar o HOG para detecção de pessoas
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    
    # Para cada frame do vídeo cortado entre ini e fin:
        # Detectar a(s) pessoa(s) no frame
        # Para cada pessoa detectada:
            # Extrair a região do braço (usar proporções do corpo ou pontos-chave)
            # Aplicar segmentação para isolar a pessoa do fundo (usar o código acima como base)
    
    # Carregar o modelo 3D da águia (Open3D?) ou usar uma imagem 2D com fundo transparente (N sei qual é suposto/melhor)
    # Posicionar a águia no braço da pessoa (usar detecção de pose ou pontos-chave)
    
    # Renderizar a águia sobre o vídeo original
    # Salvar o vídeo processado como video1.mp4
    # Retorna uma String com o nome do arquivo de vídeo processado
    return "video1.avi"

video0 = "./video0.avi"
ini = 0 # Frame inicial
fin = 10 # Frame final
video1 = AugmentedReality(video0 ,ini, fin)
